# Read in results

In [1]:
import pandas as pd
# import bioframe as bf
import numpy as np
import os
import sys

In [2]:
DATA_DIR = os.path.dirname(os.getcwd()) + '/data/experimental_maps/'

In [3]:
score_types = ['correlation', 'mse', 'ssim_map', 
               'Arrowhead',
               'chess_ssim',
               'contact_directionality_corr', 
               'dcHiC_spearman',
               'dcHiC_mse',
               'distance_enrichment_corr',
               'eigenvector_corr',
               'HiC1Dmetrics_CD',
               'HiC1Dmetrics_CIC', 
               'HiC1Dmetrics_deltaDLR',
               'HiC1Dmetrics_ISC', 
               'HiC1Dmetrics_SSC',
               'HiCcompare',
               'insulation_corr', 
               'insulation_mse',
               'loops',
               'scc',
               'Selfish',
               'TADcompare',
               'tads',
               'triangle_corr',
               'triangle_mse']

corr_scores = ['correlation', 'ssim_map', 'scc', 
               'insulation_corr', 
               'contact_directionality_corr', 
               'distance_enrichment_corr', 
               'triangle_corr', 
               'eigenvector_corr',
               'chess_ssim',
               'dcHiC_spearman',
               'HiC1Dmetrics_CD',
               'tads', 
               'loops',
               'Arrowhead']

# Read in results

In [4]:
# Methods that use contact maps
all_scores = pd.read_csv(f'{DATA_DIR}example_output/methods_that_use_contact_maps.tsv', sep = '\t')
all_scores

,chrom,start,end,correlation,mse,ssim_map,contact_directionality_corr,distance_enrichment_corr,eigenvector_corr,insulation_corr,insulation_mse,loops,scc,tads,triangle_corr,triangle_mse
0,chr22,28383571,29383571,0.145168,0.092834,0.386016,0.249530,0.176503,0.064280,0.307072,0.175929,0.153419,0.191720,0.317841,0.083702,0.087036
1,chr22,49353843,50353843,0.233731,0.069534,0.552970,0.226205,0.303671,0.052559,0.450313,0.038116,0.762190,0.299450,0.344828,0.083416,0.029279
2,chr22,40575238,41575238,0.372770,0.142020,0.475875,0.390414,0.164435,0.791050,0.003712,0.019327,0.282844,0.429492,0.137931,0.225480,0.082207
3,chr22,30846345,31846345,0.440313,0.101479,0.671219,0.554168,0.228919,0.572787,0.169129,0.066542,0.342045,0.508216,0.344828,0.282154,0.086710
4,chr22,19615937,20615937,0.284830,0.064410,0.392169,0.170340,0.269471,0.090463,0.182186,0.110110,0.350954,0.397863,0.431673,0.039809,0.075885
5,chr21,33142399,34142399,0.237529,0.115914,0.463288,0.209553,0.125526,0.211728,0.230058,0.086326,0.499630,0.355486,0.092954,0.065679,0.078323
6,chr22,42610749,43610749,0.103652,0.224987,0.484513,0.602114,0.073410,0.097498,0.638829,0.126212,0.718572,0.209457,0.392804,0.072053,0.367226
7,chr21,45073364,46073364,0.156941,0.152839,0.442794,0.386818,0.230430,0.555750,0.359251,0.185351,0.142233,0.239479,0.092954,0.202155,0.179049
8,chr22,36007220,37007220,0.133812,0.117251,0.342149,0.086142,0.167694,0.130365,0.251360,0.215512,0.117490,0.247511,0.511494,0.042428,0.086525
9,chr21,33415533,34415533,0.046555,0.054882,0.118070,0.112840,0.144245,0.107646,0.238256,0.102001,0.293638,0.104964,0.224138,0.033790,0.027777


In [5]:
# Arrowhead
arrowhead_results = (pd.read_csv(f'{DATA_DIR}example_output/arrowhead.tsv', sep = '\t', index_col = False)
             .rename(columns = {'overlap_ratio':'Arrowhead'})
            [['chrom', 'start', 'end', 'Arrowhead']])
all_scores = (all_scores.merge(arrowhead_results, on = ['chrom', 'start', 'end']))
all_scores

,chrom,start,end,correlation,mse,ssim_map,contact_directionality_corr,distance_enrichment_corr,eigenvector_corr,insulation_corr,insulation_mse,loops,scc,tads,triangle_corr,triangle_mse,Arrowhead
0,chr22,28383571,29383571,0.145168,0.092834,0.386016,0.249530,0.176503,0.064280,0.307072,0.175929,0.153419,0.191720,0.317841,0.083702,0.087036,0.454545
1,chr22,49353843,50353843,0.233731,0.069534,0.552970,0.226205,0.303671,0.052559,0.450313,0.038116,0.762190,0.299450,0.344828,0.083416,0.029279,0.533333
2,chr22,40575238,41575238,0.372770,0.142020,0.475875,0.390414,0.164435,0.791050,0.003712,0.019327,0.282844,0.429492,0.137931,0.225480,0.082207,0.350000
3,chr22,30846345,31846345,0.440313,0.101479,0.671219,0.554168,0.228919,0.572787,0.169129,0.066542,0.342045,0.508216,0.344828,0.282154,0.086710,0.500000
4,chr22,19615937,20615937,0.284830,0.064410,0.392169,0.170340,0.269471,0.090463,0.182186,0.110110,0.350954,0.397863,0.431673,0.039809,0.075885,0.647059
5,chr21,33142399,34142399,0.237529,0.115914,0.463288,0.209553,0.125526,0.211728,0.230058,0.086326,0.499630,0.355486,0.092954,0.065679,0.078323,0.357143
6,chr22,42610749,43610749,0.103652,0.224987,0.484513,0.602114,0.073410,0.097498,0.638829,0.126212,0.718572,0.209457,0.392804,0.072053,0.367226,0.545455
7,chr21,45073364,46073364,0.156941,0.152839,0.442794,0.386818,0.230430,0.555750,0.359251,0.185351,0.142233,0.239479,0.092954,0.202155,0.179049,0.611111
8,chr22,36007220,37007220,0.133812,0.117251,0.342149,0.086142,0.167694,0.130365,0.251360,0.215512,0.117490,0.247511,0.511494,0.042428,0.086525,0.800000
9,chr21,33415533,34415533,0.046555,0.054882,0.118070,0.112840,0.144245,0.107646,0.238256,0.102001,0.293638,0.104964,0.224138,0.033790,0.027777,0.307692


In [6]:
# CHESS
chess_results = (pd.read_csv(f'{DATA_DIR}example_output/CHESS.tsv', sep = '\t', index_col = False)
                 .rename(columns = {'ssim':'chess_ssim'})
                [['chrom', 'start', 'end', 'chess_ssim']])
all_scores = all_scores.merge(chess_results, on = ['chrom', 'start', 'end'])
all_scores

,chrom,start,end,correlation,mse,ssim_map,contact_directionality_corr,distance_enrichment_corr,eigenvector_corr,insulation_corr,insulation_mse,loops,scc,tads,triangle_corr,triangle_mse,Arrowhead,chess_ssim
0,chr22,28383571,29383571,0.145168,0.092834,0.386016,0.249530,0.176503,0.064280,0.307072,0.175929,0.153419,0.191720,0.317841,0.083702,0.087036,0.454545,0.545799
1,chr22,49353843,50353843,0.233731,0.069534,0.552970,0.226205,0.303671,0.052559,0.450313,0.038116,0.762190,0.299450,0.344828,0.083416,0.029279,0.533333,0.648392
2,chr22,40575238,41575238,0.372770,0.142020,0.475875,0.390414,0.164435,0.791050,0.003712,0.019327,0.282844,0.429492,0.137931,0.225480,0.082207,0.350000,0.494457
3,chr22,30846345,31846345,0.440313,0.101479,0.671219,0.554168,0.228919,0.572787,0.169129,0.066542,0.342045,0.508216,0.344828,0.282154,0.086710,0.500000,0.364405
4,chr22,19615937,20615937,0.284830,0.064410,0.392169,0.170340,0.269471,0.090463,0.182186,0.110110,0.350954,0.397863,0.431673,0.039809,0.075885,0.647059,0.876770
5,chr21,33142399,34142399,0.237529,0.115914,0.463288,0.209553,0.125526,0.211728,0.230058,0.086326,0.499630,0.355486,0.092954,0.065679,0.078323,0.357143,0.582338
6,chr22,42610749,43610749,0.103652,0.224987,0.484513,0.602114,0.073410,0.097498,0.638829,0.126212,0.718572,0.209457,0.392804,0.072053,0.367226,0.545455,0.473288
7,chr21,45073364,46073364,0.156941,0.152839,0.442794,0.386818,0.230430,0.555750,0.359251,0.185351,0.142233,0.239479,0.092954,0.202155,0.179049,0.611111,0.761645
8,chr22,36007220,37007220,0.133812,0.117251,0.342149,0.086142,0.167694,0.130365,0.251360,0.215512,0.117490,0.247511,0.511494,0.042428,0.086525,0.800000,0.649425
9,chr21,33415533,34415533,0.046555,0.054882,0.118070,0.112840,0.144245,0.107646,0.238256,0.102001,0.293638,0.104964,0.224138,0.033790,0.027777,0.307692,0.853986


In [7]:
# dcHiC
dchic_results = (pd.read_csv(f'{DATA_DIR}example_output/dcHiC.tsv', sep = '\t', index_col = False)
                 [['chrom', 'start', 'end', 'dcHiC_mse', 'dcHiC_spearman']])
all_scores = (all_scores.merge(dchic_results, on = ['chrom', 'start', 'end']))
all_scores

,chrom,start,end,correlation,mse,ssim_map,contact_directionality_corr,distance_enrichment_corr,eigenvector_corr,insulation_corr,insulation_mse,loops,scc,tads,triangle_corr,triangle_mse,Arrowhead,chess_ssim,dcHiC_mse,dcHiC_spearman
0,chr22,28383571,29383571,0.145168,0.092834,0.386016,0.249530,0.176503,0.064280,0.307072,0.175929,0.153419,0.191720,0.317841,0.083702,0.087036,0.454545,0.545799,0.842963,0.750695
1,chr22,49353843,50353843,0.233731,0.069534,0.552970,0.226205,0.303671,0.052559,0.450313,0.038116,0.762190,0.299450,0.344828,0.083416,0.029279,0.533333,0.648392,1.773680,0.253157
2,chr22,40575238,41575238,0.372770,0.142020,0.475875,0.390414,0.164435,0.791050,0.003712,0.019327,0.282844,0.429492,0.137931,0.225480,0.082207,0.350000,0.494457,0.560741,0.656379
3,chr22,30846345,31846345,0.440313,0.101479,0.671219,0.554168,0.228919,0.572787,0.169129,0.066542,0.342045,0.508216,0.344828,0.282154,0.086710,0.500000,0.364405,1.257201,0.615786
4,chr22,19615937,20615937,0.284830,0.064410,0.392169,0.170340,0.269471,0.090463,0.182186,0.110110,0.350954,0.397863,0.431673,0.039809,0.075885,0.647059,0.876770,0.665678,0.578804
5,chr21,33142399,34142399,0.237529,0.115914,0.463288,0.209553,0.125526,0.211728,0.230058,0.086326,0.499630,0.355486,0.092954,0.065679,0.078323,0.357143,0.582338,3.813881,0.368508
6,chr22,42610749,43610749,0.103652,0.224987,0.484513,0.602114,0.073410,0.097498,0.638829,0.126212,0.718572,0.209457,0.392804,0.072053,0.367226,0.545455,0.473288,1.232752,0.388758
7,chr21,45073364,46073364,0.156941,0.152839,0.442794,0.386818,0.230430,0.555750,0.359251,0.185351,0.142233,0.239479,0.092954,0.202155,0.179049,0.611111,0.761645,1.083111,0.123463
8,chr22,36007220,37007220,0.133812,0.117251,0.342149,0.086142,0.167694,0.130365,0.251360,0.215512,0.117490,0.247511,0.511494,0.042428,0.086525,0.800000,0.649425,0.635950,0.376386
9,chr21,33415533,34415533,0.046555,0.054882,0.118070,0.112840,0.144245,0.107646,0.238256,0.102001,0.293638,0.104964,0.224138,0.033790,0.027777,0.307692,0.853986,5.327371,0.399680


In [8]:
# HiC1Dmetrics 
HiC1Dmetrics_results = (pd.read_csv(f'{DATA_DIR}example_output/HiC1Dmetrics.tsv', sep = '\t')
                        .rename(columns = {'ISC':'HiC1Dmetrics_ISC',
                                           'CIC':'HiC1Dmetrics_CIC',
                                           'SSC':'HiC1Dmetrics_SSC',
                                           'deltaDLR':'HiC1Dmetrics_deltaDLR',
                                           'CD':'HiC1Dmetrics_CD'})
                        .sort_values(['chrom', 'start', 'end'])
                        [['chrom', 'start', 'end', 
                          'HiC1Dmetrics_ISC', 'HiC1Dmetrics_CIC', 'HiC1Dmetrics_SSC', 
                          'HiC1Dmetrics_deltaDLR', 'HiC1Dmetrics_CD']])
all_scores = (all_scores.merge(HiC1Dmetrics_results, on = ['chrom', 'start', 'end']))
all_scores

,chrom,start,end,correlation,mse,ssim_map,contact_directionality_corr,distance_enrichment_corr,eigenvector_corr,insulation_corr,...,triangle_mse,Arrowhead,chess_ssim,dcHiC_mse,dcHiC_spearman,HiC1Dmetrics_ISC,HiC1Dmetrics_CIC,HiC1Dmetrics_SSC,HiC1Dmetrics_deltaDLR,HiC1Dmetrics_CD
0,chr22,28383571,29383571,0.145168,0.092834,0.386016,0.249530,0.176503,0.064280,0.307072,...,0.087036,0.454545,0.545799,0.842963,0.750695,0.057142,0.259696,0.059672,0.554394,0.991991
1,chr22,49353843,50353843,0.233731,0.069534,0.552970,0.226205,0.303671,0.052559,0.450313,...,0.029279,0.533333,0.648392,1.773680,0.253157,0.048283,0.367519,0.073648,0.831367,0.991229
2,chr22,40575238,41575238,0.372770,0.142020,0.475875,0.390414,0.164435,0.791050,0.003712,...,0.082207,0.350000,0.494457,0.560741,0.656379,0.039727,0.244202,0.057153,0.533301,0.992172
3,chr22,30846345,31846345,0.440313,0.101479,0.671219,0.554168,0.228919,0.572787,0.169129,...,0.086710,0.500000,0.364405,1.257201,0.615786,0.051692,0.244422,0.053920,0.518345,0.990931
4,chr22,19615937,20615937,0.284830,0.064410,0.392169,0.170340,0.269471,0.090463,0.182186,...,0.075885,0.647059,0.876770,0.665678,0.578804,0.061606,0.284792,0.057846,0.605862,0.991545
5,chr21,33142399,34142399,0.237529,0.115914,0.463288,0.209553,0.125526,0.211728,0.230058,...,0.078323,0.357143,0.582338,3.813881,0.368508,0.052756,0.247882,0.058159,0.478409,0.992025
6,chr22,42610749,43610749,0.103652,0.224987,0.484513,0.602114,0.073410,0.097498,0.638829,...,0.367226,0.545455,0.473288,1.232752,0.388758,0.060225,0.328559,0.068272,0.729356,0.991018
7,chr21,45073364,46073364,0.156941,0.152839,0.442794,0.386818,0.230430,0.555750,0.359251,...,0.179049,0.611111,0.761645,1.083111,0.123463,0.054493,0.305786,0.065250,0.629245,0.990239
8,chr22,36007220,37007220,0.133812,0.117251,0.342149,0.086142,0.167694,0.130365,0.251360,...,0.086525,0.800000,0.649425,0.635950,0.376386,0.064141,0.280449,0.061449,0.565497,0.990890
9,chr21,33415533,34415533,0.046555,0.054882,0.118070,0.112840,0.144245,0.107646,0.238256,...,0.027777,0.307692,0.853986,5.327371,0.399680,0.047581,0.244026,0.057787,0.473821,0.992692


In [9]:
# Selfish
selfish_results = (pd.read_csv(f'{DATA_DIR}example_output/selfish.tsv', sep = '\t')
                   [['chrom', 'start', 'end', 'Selfish']])
all_scores = (all_scores.merge(selfish_results, on = ['chrom', 'start', 'end']))
all_scores

,chrom,start,end,correlation,mse,ssim_map,contact_directionality_corr,distance_enrichment_corr,eigenvector_corr,insulation_corr,...,Arrowhead,chess_ssim,dcHiC_mse,dcHiC_spearman,HiC1Dmetrics_ISC,HiC1Dmetrics_CIC,HiC1Dmetrics_SSC,HiC1Dmetrics_deltaDLR,HiC1Dmetrics_CD,Selfish
0,chr22,28383571,29383571,0.145168,0.092834,0.386016,0.249530,0.176503,0.064280,0.307072,...,0.454545,0.545799,0.842963,0.750695,0.057142,0.259696,0.059672,0.554394,0.991991,2558
1,chr22,49353843,50353843,0.233731,0.069534,0.552970,0.226205,0.303671,0.052559,0.450313,...,0.533333,0.648392,1.773680,0.253157,0.048283,0.367519,0.073648,0.831367,0.991229,1574
2,chr22,40575238,41575238,0.372770,0.142020,0.475875,0.390414,0.164435,0.791050,0.003712,...,0.350000,0.494457,0.560741,0.656379,0.039727,0.244202,0.057153,0.533301,0.992172,1674
3,chr22,30846345,31846345,0.440313,0.101479,0.671219,0.554168,0.228919,0.572787,0.169129,...,0.500000,0.364405,1.257201,0.615786,0.051692,0.244422,0.053920,0.518345,0.990931,1705
4,chr22,19615937,20615937,0.284830,0.064410,0.392169,0.170340,0.269471,0.090463,0.182186,...,0.647059,0.876770,0.665678,0.578804,0.061606,0.284792,0.057846,0.605862,0.991545,2548
5,chr21,33142399,34142399,0.237529,0.115914,0.463288,0.209553,0.125526,0.211728,0.230058,...,0.357143,0.582338,3.813881,0.368508,0.052756,0.247882,0.058159,0.478409,0.992025,2422
6,chr22,42610749,43610749,0.103652,0.224987,0.484513,0.602114,0.073410,0.097498,0.638829,...,0.545455,0.473288,1.232752,0.388758,0.060225,0.328559,0.068272,0.729356,0.991018,2512
7,chr21,45073364,46073364,0.156941,0.152839,0.442794,0.386818,0.230430,0.555750,0.359251,...,0.611111,0.761645,1.083111,0.123463,0.054493,0.305786,0.065250,0.629245,0.990239,3265
8,chr22,36007220,37007220,0.133812,0.117251,0.342149,0.086142,0.167694,0.130365,0.251360,...,0.800000,0.649425,0.635950,0.376386,0.064141,0.280449,0.061449,0.565497,0.990890,3702
9,chr21,33415533,34415533,0.046555,0.054882,0.118070,0.112840,0.144245,0.107646,0.238256,...,0.307692,0.853986,5.327371,0.399680,0.047581,0.244026,0.057787,0.473821,0.992692,2406


In [10]:
# TADcompare and HiCcompare 

# HiCcompare files are too large to upload on github so they are not included and will need to be generated

for chrom in ['chr21', 'chr22']:
    
    # TADcompare
    TADcompare_chr = pd.read_csv(f'{DATA_DIR}example_output/TADcompare_{chrom}', sep ='\t')
    TADcompare_chr['score'] = 1 # set score to 1 for any type of TAD change
    TADcompare_chr.loc[TADcompare_chr.Differential == 'Non-Differential', 'score'] = 0
    
    
    # HiCcompare
    HiCcompare_chr = pd.read_csv(f'{DATA_DIR}example_output/HiCcompare_{chrom}', sep ='\t')
    HiCcompare_chr['score'] = 0
    HiCcompare_chr.loc[HiCcompare_chr['p.adj'] < 0.05, 'score'] = 1 # set score to 1 for significant changes
    
    
    # Sum all scores in the window
    for i in all_scores.index:
    
        start = all_scores.loc[i,'start']
        end = all_scores.loc[i,'end']
    
        all_scores.loc[i,'TADcompare'] = (TADcompare_chr
                                                     .query('Boundary > @start & Boundary < @end')
                                                     .score
                                                     .sum())
        all_scores.loc[i,'HiCcompare'] = (HiCcompare_chr
                                                     .query('start1 > @start & end2 < @end')
                                                     .score
                                                     .sum())
all_scores

,chrom,start,end,correlation,mse,ssim_map,contact_directionality_corr,distance_enrichment_corr,eigenvector_corr,insulation_corr,...,dcHiC_mse,dcHiC_spearman,HiC1Dmetrics_ISC,HiC1Dmetrics_CIC,HiC1Dmetrics_SSC,HiC1Dmetrics_deltaDLR,HiC1Dmetrics_CD,Selfish,TADcompare,HiCcompare
0,chr22,28383571,29383571,0.145168,0.092834,0.386016,0.249530,0.176503,0.064280,0.307072,...,0.842963,0.750695,0.057142,0.259696,0.059672,0.554394,0.991991,2558,12.0,1.0
1,chr22,49353843,50353843,0.233731,0.069534,0.552970,0.226205,0.303671,0.052559,0.450313,...,1.773680,0.253157,0.048283,0.367519,0.073648,0.831367,0.991229,1574,10.0,8.0
2,chr22,40575238,41575238,0.372770,0.142020,0.475875,0.390414,0.164435,0.791050,0.003712,...,0.560741,0.656379,0.039727,0.244202,0.057153,0.533301,0.992172,1674,8.0,0.0
3,chr22,30846345,31846345,0.440313,0.101479,0.671219,0.554168,0.228919,0.572787,0.169129,...,1.257201,0.615786,0.051692,0.244422,0.053920,0.518345,0.990931,1705,7.0,1.0
4,chr22,19615937,20615937,0.284830,0.064410,0.392169,0.170340,0.269471,0.090463,0.182186,...,0.665678,0.578804,0.061606,0.284792,0.057846,0.605862,0.991545,2548,23.0,5.0
5,chr21,33142399,34142399,0.237529,0.115914,0.463288,0.209553,0.125526,0.211728,0.230058,...,3.813881,0.368508,0.052756,0.247882,0.058159,0.478409,0.992025,2422,2.0,0.0
6,chr22,42610749,43610749,0.103652,0.224987,0.484513,0.602114,0.073410,0.097498,0.638829,...,1.232752,0.388758,0.060225,0.328559,0.068272,0.729356,0.991018,2512,12.0,1.0
7,chr21,45073364,46073364,0.156941,0.152839,0.442794,0.386818,0.230430,0.555750,0.359251,...,1.083111,0.123463,0.054493,0.305786,0.065250,0.629245,0.990239,3265,18.0,0.0
8,chr22,36007220,37007220,0.133812,0.117251,0.342149,0.086142,0.167694,0.130365,0.251360,...,0.635950,0.376386,0.064141,0.280449,0.061449,0.565497,0.990890,3702,17.0,0.0
9,chr21,33415533,34415533,0.046555,0.054882,0.118070,0.112840,0.144245,0.107646,0.238256,...,5.327371,0.399680,0.047581,0.244026,0.057787,0.473821,0.992692,2406,2.0,0.0


# Normalize scores

In [11]:
# Normalize scores (min/max norm to 0-1)

all_scores_scaled = all_scores.copy(deep=True)

for score in score_types:

    data = all_scores_scaled[score]

    if score in ['eigenvector_corr', 'dcHiC_spearman']:
        data = [abs(x) for x in data]
        
    if score in corr_scores:
        data = [1 - x for x in data]

    data = [(x - np.nanmin(data)) / (np.nanmax(data) - np.nanmin(data)) for x in data]
    
    all_scores_scaled[score] = data

all_scores_scaled

,chrom,start,end,correlation,mse,ssim_map,contact_directionality_corr,distance_enrichment_corr,eigenvector_corr,insulation_corr,...,dcHiC_mse,dcHiC_spearman,HiC1Dmetrics_ISC,HiC1Dmetrics_CIC,HiC1Dmetrics_SSC,HiC1Dmetrics_deltaDLR,HiC1Dmetrics_CD,Selfish,TADcompare,HiCcompare
0,chr22,28383571,29383571,0.749558,0.223106,0.515600,0.683340,0.552280,0.984128,0.522357,...,0.059208,0.000000,0.713324,0.126885,0.291557,0.225350,0.285646,0.462406,0.476190,0.125
1,chr22,49353843,50353843,0.524641,0.086132,0.213774,0.728546,0.000000,1.000000,0.296821,...,0.254465,0.793228,0.350461,1.000000,1.000000,1.000000,0.596584,0.000000,0.380952,1.000
2,chr22,40575238,41575238,0.171533,0.512260,0.353150,0.410294,0.604691,0.000000,1.000000,...,0.000000,0.150369,0.000000,0.001423,0.163897,0.166355,0.211843,0.046992,0.285714,0.000
3,chr22,30846345,31846345,0.000000,0.273932,0.000000,0.092922,0.324639,0.295554,0.739550,...,0.146111,0.215086,0.490100,0.003207,0.000000,0.124525,0.717841,0.061560,0.238095,0.125
4,chr22,19615937,20615937,0.394868,0.056013,0.504476,0.836817,0.148527,0.948674,0.718991,...,0.022015,0.274047,0.896156,0.330106,0.199012,0.369298,0.467701,0.457707,1.000000,0.625
5,chr21,33142399,34142399,0.514996,0.358789,0.375905,0.760818,0.773665,0.784467,0.643616,...,0.682482,0.609324,0.533687,0.031226,0.214897,0.012831,0.271918,0.398496,0.000000,0.000
6,chr22,42610749,43610749,0.854995,1.000000,0.337534,0.000000,1.000000,0.939148,0.000000,...,0.140982,0.577038,0.839608,0.684512,0.727497,0.714691,0.682567,0.440789,0.476190,0.125
7,chr21,45073364,46073364,0.719659,0.575860,0.412954,0.417263,0.318078,0.318623,0.440199,...,0.109589,1.000000,0.604827,0.500110,0.574317,0.434697,1.000000,0.794643,0.761905,0.000
8,chr22,36007220,37007220,0.778399,0.366647,0.594904,1.000000,0.590536,0.894642,0.610076,...,0.015778,0.596763,1.000000,0.294935,0.381620,0.256402,0.734531,1.000000,0.714286,0.000
9,chr21,33415533,34415533,1.000000,0.000000,1.000000,0.948257,0.692373,0.925407,0.630707,...,1.000000,0.559625,0.321703,0.000000,0.196038,0.000000,0.000000,0.390977,0.000000,0.000
